In [1]:
import numpy as np
import pandas as pd
import datetime
import time

import lightgbm as lgb


from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score

import itertools

import warnings
import json

warnings.filterwarnings('ignore')


df = pd.read_csv(r'./train_data.csv')
df.head(5)

,企业编号,企业总评分,软著数量,作品著作数量,项目数量,纳税A级年份_2014,纳税A级年份_2015,纳税A级年份_2016,纳税A级年份_2017,纳税A级年份增长1,...,应收账款周转天数(天)_mean,应收账款周转天数(天)_max,应收账款周转天数(天)_min,应收账款周转天数(天)_std,应收账款周转天数(天)滚动增长_mean,存货周转天数(天)_mean,存货周转天数(天)_max,存货周转天数(天)_min,存货周转天数(天)_std,存货周转天数(天)滚动增长_mean
0,1001,75.374276,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,...,107.589270,191.707773,63.791689,44.495607,0.151392,414.778035,1089.655763,176.283983,325.371499,1.562757
1,1002,79.830122,2.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,...,46.903333,56.590000,39.830000,6.234116,0.023916,6.506667,7.040000,5.010000,0.702335,0.045330
2,1003,78.318264,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,84.275556,139.910000,56.020000,33.143654,-0.040224,54.918889,75.540000,38.010000,11.089465,0.031792
3,1004,83.253376,0.0,6.0,1.0,0.0,0.0,2.0,1.0,0.0,...,26.720000,35.360000,17.290000,6.024438,0.081857,6.954444,7.900000,6.240000,0.618448,0.021711
4,1005,83.291493,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,94.050000,110.260000,77.850000,9.652235,0.012921,108.584444,357.190000,44.160000,101.728838,0.344086


# Train data

In [2]:
y = df[['企业编号', '企业总评分']]
x = df.drop(['企业总评分'], axis=1)

xtrain, xtest, ytrain, ytest = train_test_split(
    x, y, test_size=0.2, random_state=31)
ytrain_id = ytrain['企业编号']
ytrain = ytrain['企业总评分']
ytest_id = ytest['企业编号']
ytest = ytest['企业总评分']
print(xtrain.shape, xtest.shape, ytrain.shape, ytest.shape)

id_train = xtrain['企业编号']
id_test = xtest['企业编号']
xtrain.drop(['企业编号'], axis=1, inplace=True)
xtest.drop(['企业编号'], axis=1, inplace=True)
feature_name = xtrain.columns.values

(2364, 335) (592, 335) (2364,) (592,)


In [3]:
estimator = lgb.LGBMRegressor(objective='regression',
                              num_leaves=20,
                              learning_rate=0.005,
                              n_estimators=1500,
                              max_depth=7,
                              boosting='gbdt',
                              metric='rmse',
                              max_bin=35,
                              n_jobs=-1,
                              min_child_samples=100,
                              bagging_seed=11,
                              bagging_fraction=0.7,
                              bagging_freq=1,
                              min_data_in_leaf=20
                              )

# RFE

In [0]:
for i in range(334, 120, -5):
    selector = RFE(estimator, i, step=1)
    selector = selector.fit(xtrain, ytrain)
    pre = selector.predict(xtest)
    print(i, np.sqrt(mean_squared_error(pre, ytest)))
# The best is 124

In [4]:
selector = RFE(estimator, 124, step=1)
selector = selector.fit(xtrain, ytrain)
pre = selector.predict(xtest)
print(np.sqrt(mean_squared_error(pre, ytest)), r2_score(pre, ytest))

3.0914263498786956 0.42624261246261586


# Test data

In [5]:
test_df = pd.read_csv(r'./test_data.csv')
X_test = test_df.drop(['企业编号'], axis=1)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test = X_test.fillna(0)
test_id = test_df['企业编号']

In [6]:
test_df.head(5)

,企业编号,软著数量,作品著作数量,项目数量,纳税A级年份_2014,纳税A级年份_2015,纳税A级年份_2016,纳税A级年份_2017,纳税A级年份增长1,纳税A级年份增长2,...,应收账款周转天数(天)_mean,应收账款周转天数(天)_max,应收账款周转天数(天)_min,应收账款周转天数(天)_std,应收账款周转天数(天)滚动增长_mean,存货周转天数(天)_mean,存货周转天数(天)_max,存货周转天数(天)_min,存货周转天数(天)_std,存货周转天数(天)滚动增长_mean
0,4001,188.0,12.0,1.0,1.0,1.0,2.0,0.0,0.0,1.0,...,-0.053552,853.253333,1078.28,647.43,132.634622,0.012903,0.0,0.0,0.0,0.0
1,4002,42.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,-0.053116,100.236667,160.28,69.50,31.849951,-0.045968,0.0,0.0,0.0,0.0
2,4003,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,...,0.023584,45.646111,63.87,25.49,11.654292,0.031598,0.0,0.0,0.0,0.0
3,4004,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.012479,1.691111,2.17,1.45,0.212394,0.048454,0.0,0.0,0.0,0.0
4,4005,19.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,-1.0,...,-0.097558,616.920000,904.24,329.91,191.017424,-0.054086,0.0,0.0,0.0,0.0


In [7]:
pre = selector.predict(X_test)

In [29]:
pre

array([90.77802032, 88.63823513, 81.2188751 , 79.55067551, 88.91477832,
       82.75860031, 85.67400315, 85.54302917, 83.77771987, 86.21060123,
       86.605372  , 83.00275649, 81.48670908, 81.65139362, 84.12174345,
       84.62060389, 86.04768756, 85.30019194, 84.60560685, 87.10072435,
       84.53515458, 80.44381285, 82.7006676 , 84.73956797, 85.26982366,
       82.01186393, 81.11491703, 80.32427226, 84.43252362, 82.47032861,
       84.56428188, 78.62224408, 84.97578661, 87.95468996, 82.74784602,
       87.38528768, 81.3220463 , 84.35176437, 81.12684509, 83.87039829,
       83.86520632, 80.44961347, 80.58680561, 83.2308124 , 81.22325753,
       84.96372232, 88.15350749, 82.55453637, 82.92442939, 84.35188388,
       83.403397  , 87.66743141, 80.20242315, 86.59796346, 85.73295898,
       82.17495475, 86.04821792, 85.49279881, 83.47177898, 86.41876697,
       83.6772916 , 86.98257265, 88.408019  , 82.66224587, 84.36776631,
       84.36825004, 83.23733396, 83.47359347, 86.89021767, 83.35

In [8]:
result = pd.DataFrame({
    '企业编号': test_id,
    'pre_rating': pd.Series(pre)
})

In [32]:
result.to_excel(r'./赛题 1 结果_Trainee.xlsx', index=False, header=False)

# Result Validation

In [17]:
# 可以看出，官方提供的得分并不是整数
real_rating = pd.read_excel('./Mission1_Real.xlsx', header=None)
real_rating.columns = ['企业编号', 'pre_rating']
real_rating.head()

,企业编号,pre_rating
0,4001,93.076923
1,4002,93.907074
2,4003,82.012192
3,4004,81.780574
4,4005,91.538988


In [15]:
# result.shape
# real_rating.shape

(520, 2)

In [20]:
np.sqrt(mean_squared_error(result['pre_rating'].astype(int).values,
                           real_rating['pre_rating'].astype(int).values))

3.69927227770752

In [23]:
pre_data = pd.read_excel(r'./赛题 1 结果_Trainee.xlsx', header=None)
pre_data.columns = ['企业编号', 'pre_rating']
pre_data.head()

,企业编号,pre_rating
0,4001,90
1,4002,88
2,4003,81
3,4004,79
4,4005,88


In [31]:
# 重新跑了一次模型，参数未变，模型未变，取模型预测值（未取整数），与官方提供的真实得分（未取整数）做rmse
np.sqrt(mean_squared_error(pre,
                           real_rating['pre_rating'].values))
# 得分为3.69

3.693318683603019

In [39]:
# 重新跑了一次模型，参数未变，模型未变，取模型预测值（未取整数），与官方提供的真实得分（取整数）做rmse
np.sqrt(mean_squared_error(pre,
                           real_rating['pre_rating'].astype(int).values))
# 得分为3.60

3.600994180484945

In [24]:
# 取Trainee团队提交的成绩，取整数，与官方提供的真实得分（取整数）做rmse
np.sqrt(mean_squared_error(pre_data['pre_rating'].astype(int).values,
                           real_rating['pre_rating'].astype(int).values))
# 得分为3.69

3.69927227770752

In [25]:
# 取Trainee团队提交的成绩，取整数，与官方提供的真实得分（未取整数）做rmse
np.sqrt(mean_squared_error(pre_data['pre_rating'].values,
                           real_rating['pre_rating'].values))
# 得分未3.85， 这分数才对应上官方告知Trainee团队的比赛得分

3.855974497318089

In [28]:
pre_data.head()

,企业编号,pre_rating
0,4001,90
1,4002,88
2,4003,81
3,4004,79
4,4005,88


# Feature Validation

In [33]:
estimator_v = lgb.LGBMRegressor(objective='regression',
                                num_leaves=20,
                                learning_rate=0.005,
                                n_estimators=1500,
                                max_depth=7,
                                boosting='gbdt',
                                metric='rmse',
                                max_bin=35,
                                n_jobs=-1,
                                min_child_samples=100,
                                bagging_seed=11,
                                bagging_fraction=0.7,
                                bagging_freq=1,
                                min_data_in_leaf=20
                                )

In [34]:
estimator_v.fit(xtrain, ytrain)

LGBMRegressor(bagging_fraction=0.7, bagging_freq=1, bagging_seed=11,
       boosting='gbdt', boosting_type='gbdt', class_weight=None,
       colsample_bytree=1.0, importance_type='split', learning_rate=0.005,
       max_bin=35, max_depth=7, metric='rmse', min_child_samples=100,
       min_child_weight=0.001, min_data_in_leaf=20, min_split_gain=0.0,
       n_estimators=1500, n_jobs=-1, num_leaves=20, objective='regression',
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [35]:
pre_v = estimator_v.predict(X_test)

In [36]:
np.sqrt(mean_squared_error(pre_v,
                           real_rating['pre_rating'].values))

3.6519087051653103

In [37]:
np.sqrt(mean_squared_error(pre_v.astype(int),
                           real_rating['pre_rating'].values))

3.831554603992627

In [38]:
np.sqrt(mean_squared_error(pre_v.astype(int),
                           real_rating['pre_rating'].astype(int).values))

3.6948508869673047